In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook-unmodified.db

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook-unmodified.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [5]:
%%sql

WITH 
usa_tracks AS
(
    SELECT g.name, t.track_id
    FROM invoice i
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    WHERE Billing_Country = 'USA'
)

SELECT 
    name AS genre, 
    count(track_id) AS tracks_sold, 
    ROUND(CAST(count(track_id) AS FLOAT) / (SELECT COUNT(*) FROM usa_tracks), 2) as percentage_sold
FROM usa_tracks
GROUP BY name
ORDER BY 2 DESC

 * sqlite:///chinook-unmodified.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Pop,22,0.02
Latin,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


Based on the data in the graphs, the new record label should add Red Tone (punk), Slim Jim Bites (blues), Meteor and the Girl (pop), and Regal (hip-hop) in order of prefrance

In [6]:
%%sql
SELECT 
    e.first_name || " " || e.last_name AS Employee,
    '$' || ROUND(SUM(total), 2) AS 'Total Sales'
FROM employee e
LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY e.employee_id


 * sqlite:///chinook-unmodified.db
Done.


Employee,Total Sales
Andrew Adams,None
Nancy Edwards,None
Jane Peacock,$1731.51
Margaret Park,$1584.0
Steve Johnson,$1393.92
Michael Mitchell,None
Robert King,None
Laura Callahan,None


NOTE: 
Michael Mitchel, Robert King, and Laura Callahan are 
IT employees and are not involved in sales. 
Adam Andrew and Nancy Edwards are also managers so they
are also not involved with sales

In [7]:
%%sql

WITH 
t AS 
(
    SELECT 
        c.country, 
        COUNT(DISTINCT(c.customer_id)) AS total_customers,
        SUM(i.total) AS total_sales,
        COUNT(i.invoice_id) AS num_invoices,
        SUM(i.total) / COUNT(DISTINCT(c.customer_id)) AS avg_sales_per_customer,
        SUM(i.total) / COUNT(i.invoice_id) AS avg_order_value,
    CASE
        WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 'Other'
        ELSE country
        END 
        AS new_country
    FROM customer c
    LEFT JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY country
    ORDER BY 2 DESC
),
t_sort AS
(
    SELECT
        new_country,
        SUM(total_customers) total_customers,
        ROUND(SUM(total_sales), 1) total_sales, 
        ROUND(SUM(total_sales) / SUM(total_customers), 1) avg_sales_per_customer,
        ROUND(SUM(total_sales) / SUM(num_invoices), 1) avg_order_value
    FROM t
    GROUP BY new_country
    ORDER BY 3 DESC
)

SELECT
    new_country,
    total_customers,
    total_sales,
    avg_sales_per_customer,
    avg_order_value
FROM 
    (
    SELECT 
     ts.*,
    CASE
        WHEN new_country = "Other" THEN 1
        ELSE 0
        END AS sort  
    FROM t_sort as ts
    )
ORDER BY sort ASC

 * sqlite:///chinook-unmodified.db
Done.


new_country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.5,80.0,7.9
Canada,8,535.6,66.9,7.0
Brazil,5,427.7,85.5,7.0
France,5,389.1,77.8,7.8
Germany,4,334.6,83.7,8.2
Czech Republic,2,273.2,136.6,9.1
United Kingdom,3,245.5,81.8,8.8
Portugal,2,185.1,92.6,6.4
India,2,183.2,91.6,8.7
Other,15,1094.9,73.0,7.4


USA has the most amount of customers, followed by Canada and Brazil

In [3]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook-unmodified.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


From the information above, it is clear that the majority of customers do not by the full album. Thus, the better puchasing strategy is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. 